In [1]:
# import required packages
import pandas as pd
from collections import defaultdict
from tqdm import tqdm

In [2]:
# config
ds_name = 'Bookcrossing'
ds_acronym = 'BX'
ds_core = '5Rate'
ds_feedback = 'Explicit'

In [3]:
# loading the Book-Crossing dataset
dataset = pd.read_csv(f"dataset/{ds_name}/{ds_acronym}-Book-{ds_feedback}-{ds_core}-Map.csv", sep=",")

In [17]:
user_books = defaultdict(list)
for eachline in tqdm(dataset.iterrows()):
    uid, bid, rating = eachline[1][0], eachline[1][1], eachline[1][2] 
    uid, bid = int(uid), int(bid)
    if uid in user_books.keys():
        user_books[uid].append(bid)
    else:
        user_books[uid] = [bid]

88552it [00:11, 7613.84it/s]


In [21]:
def group_profile_size():
    for group in ["active_users", "medium_users", "inactive_users"]:
        count = 0
        user_group = pd.read_csv(f"dataset/Bookcrossing/user-groups/{group}.txt")
        for eachline in user_group.itertuples(index=True):
            count += len(user_books[eachline.user_id])
        print(f"{group}: {count}")

In [22]:
group_profile_size()

active_users: 12657
medium_users: 61889
inactive_users: 14006
